In [1]:
import sys, os
from Deconvolution.BLADE import Framework
import numpy as np
from numpy import transpose as t
import itertools
import pickle
from scipy.optimize import nnls
from sklearn.svm import SVR
from sklearn.svm import NuSVR

from sklearn.metrics import mean_squared_error as mse
import pandas as pd

# modules for visualization
import qgrid
from matplotlib import pyplot as plt
import seaborn as sns

# pseudobulk analysis


In [11]:
labels = pd.read_csv("/home/cke/Puram/PuramHNSCC_cell_category_mappedCelltypes_simple_symbol.csv",index_col=0)

In [12]:
labels

,cell_category
HN28_P15_D06_S330_comb,Fibroblast
HN28_P6_G05_S173_comb,Fibroblast
HN26_P14_D11_S239_comb,tumor
HN26_P14_H05_S281_comb,Fibroblast
HN26_P25_H09_S189_comb,tumor
...,...
HNSCC17_P4_G12_S180_comb,other
HNSCC20_P13_C05_S221_comb,tumor
HNSCC17_P4_C12_S132_comb,tumor
HNSCC20_P3_H08_S92_comb,tumor


In [13]:
dict_label = labels.to_dict()['cell_category']
dict_label

{'HN28_P15_D06_S330_comb': 'Fibroblast',
 'HN28_P6_G05_S173_comb': 'Fibroblast',
 'HN26_P14_D11_S239_comb': 'tumor',
 'HN26_P14_H05_S281_comb': 'Fibroblast',
 'HN26_P25_H09_S189_comb': 'tumor',
 'HN26_P14_H06_S282_comb': 'tumor',
 'HN25_P25_C04_S316_comb': 'tumor',
 'HN26_P25_A11_S107_comb': 'Fibroblast',
 'HN26_P25_C09_S129_comb': 'tumor',
 'HNSCC26_P24_H05_S377_comb': 'tumor',
 'HN25_P5_E10_S58_comb': 'Fibroblast',
 'HN25_P24_B02_S110_comb': 'tumor',
 'HN25_P25_C01_S313_comb': 'B cell',
 'HN28_P15_G06_S366_comb': 'Fibroblast',
 'HN26_P6_B06_S18_comb': 'Fibroblast',
 'HN25_P5_C05_S29_comb': 'Fibroblast',
 'HN28_P6_D09_S141_comb': 'Fibroblast',
 'HN26_P5_H01_S85_comb': 'tumor',
 'HN26_P13_D07_S139_comb': 'Fibroblast',
 'HN26_P5_E06_S54_comb': 'tumor',
 'HN26_P26_H05_S377_comb': 'tumor',
 'HNSCC26_P24_F12_S360_comb': 'tumor',
 'HNSCC26_P24_F03_S351_comb': 'tumor',
 'HN26_P6_D08_S44_comb': 'tumor',
 'HN25_P5_E12_S60_comb': 'Fibroblast',
 'HNSCC26_P24_C01_S313_comb': 'tumor',
 'HN25_P5_F0

In [59]:
filename = "/home/cke/Puram/Puram_pseudobulk_scRNAref_100DEGs_signature.tsv"
scRNAref =  pd.read_csv(filename,index_col=0)

In [60]:
scRNAref.columns = scRNAref.columns.map(dict_label)

In [61]:
scRNAref

,Fibroblast,Fibroblast,tumor,Fibroblast,tumor,B cell,Fibroblast,Fibroblast,Fibroblast,Fibroblast,...,tumor,tumor,tumor,tumor,tumor,tumor,tumor,tumor,tumor,tumor
WDR66,0.000000,0.000000,70.970700,0.000000,21.619180,0.000000,0.000000,0.000000,0.000000,0.000000,...,88.109985,186.696580,48.709743,186.751130,94.360220,350.468170,60.969543,106.592255,32.998910,129.810420
CST3,338.237580,456.304350,1289.218300,1413.501300,926.949900,0.000000,362.301640,2177.895500,550.872130,1086.376800,...,164.505430,368.468140,222.845320,253.658750,483.986600,342.976440,176.913650,794.878970,272.602050,81.109406
ACTN2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CPEB4,30.370459,26.789652,0.000000,11.710606,9.230017,34.710075,0.000000,22.709280,0.000000,21.889910,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
TMEM176B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.119827,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ADRBK2,0.360000,1.239984,0.789964,0.000000,0.210001,0.889970,0.000000,0.480002,0.000000,0.000000,...,0.889970,0.669999,0.000000,0.000000,1.029979,0.000000,0.610003,1.029979,0.550000,17.889072
CD84,0.819996,0.000000,6.749987,2.629959,0.000000,0.949993,0.969973,1.829999,0.960017,0.620002,...,0.819996,0.370001,0.419998,1.110009,0.419998,1.359978,0.279998,0.569997,0.540001,0.390002
FAM105A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.819138,0.000000,0.000000,...,0.709996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ZEB2,0.000000,0.000000,0.000000,67.769260,0.000000,0.000000,0.000000,0.880011,0.000000,67.398200,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [62]:
scRNAref.to_csv(filename.split(".")[0]+"_toCIBERSORTx.tsv",sep='\t')